# Permuted Pixel MNIST Demo 
Light weighted demo of our DilatedRNN on Pixel MNist with permutation.

In [1]:
import sys
sys.path.append("./models")
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from classification_models import drnn_classification

In [2]:
# configurations
data_dir = "./MNIST_data"
n_steps = 28*28
input_dims = 1
n_classes = 10 

# model config
cell_type = "LSTM"
assert(cell_type in ["RNN", "LSTM", "GRU"])
hidden_structs = [20] * 9
dilations = [1, 2, 4, 8, 16, 32, 64, 128, 256]
assert(len(hidden_structs) == len(dilations))

# learning config
batch_size = 128
learning_rate = 1.0e-3
training_iters = batch_size * 30000
testing_step = 5000
display_step = 100

# permutation seed 
seed = 92916

In [3]:
mnist = input_data.read_data_sets(data_dir, one_hot=True)
if 'seed' in globals():
    # randomly permute np.arange(n_steps), seed is the state, if the state is the same, the result is the same 
    rng_permute = np.random.RandomState(seed)
    idx_permute = rng_permute.permutation(n_steps)
else:
    idx_permute = np.random.permutation(n_steps)

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
# build computation graph
tf.reset_default_graph()
x = tf.placeholder(tf.float32, [None, n_steps, input_dims])
y = tf.placeholder(tf.float32, [None, n_classes])    
global_step = tf.Variable(0, name='global_step', trainable=False)

# build prediction graph
print ("==> Building a dRNN with %s cells" %cell_type)
pred = drnn_classification(x, hidden_structs, dilations, n_steps, n_classes, input_dims, cell_type)

# build loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate, 0.9).minimize(cost, global_step=global_step)

# evaluation model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

==> Building a dRNN with LSTM cells
Building layer: multi_dRNN_dilation_1, input length: 784, dilation rate: 1, input dim: 1.
=====> Input length for sub-RNN: 784
Building layer: multi_dRNN_dilation_2, input length: 784, dilation rate: 2, input dim: 20.
=====> Input length for sub-RNN: 392
Building layer: multi_dRNN_dilation_4, input length: 784, dilation rate: 4, input dim: 20.
=====> Input length for sub-RNN: 196
Building layer: multi_dRNN_dilation_8, input length: 784, dilation rate: 8, input dim: 20.
=====> Input length for sub-RNN: 98
Building layer: multi_dRNN_dilation_16, input length: 784, dilation rate: 16, input dim: 20.
=====> Input length for sub-RNN: 49
Building layer: multi_dRNN_dilation_32, input length: 784, dilation rate: 32, input dim: 20.
=====> 16 time points need to be padded. 
=====> Input length for sub-RNN: 25
Building layer: multi_dRNN_dilation_64, input length: 784, dilation rate: 64, input dim: 20.
=====> 48 time points need to be padded. 
=====> Input length

In [5]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [6]:
step = 0
train_results = []
validation_results = []
test_results = []

while step * batch_size < training_iters:
    batch_x, batch_y = mnist.train.next_batch(batch_size)    
    batch_x = batch_x[:, idx_permute]
    batch_x = batch_x.reshape([batch_size, n_steps, input_dims])

    feed_dict = {
        x : batch_x,
        y : batch_y
    }
    cost_, accuracy_, step_,  _ = sess.run([cost, accuracy, global_step, optimizer], feed_dict=feed_dict)    
    train_results.append((step_, cost_, accuracy_))    

    if (step + 1) % display_step == 0:
        print ("Iter " + str(step + 1) + ", Minibatch Loss: " + "{:.6f}".format(cost_) \
        + ", Training Accuracy: " + "{:.6f}".format(accuracy_))
             
    if (step + 1) % testing_step == 0:
        
        # validation performance
        batch_x = mnist.validation.images
        batch_y = mnist.validation.labels

        # permute the data
        batch_x = batch_x[:, idx_permute]        
        batch_x = batch_x.reshape([-1, n_steps, input_dims])
        feed_dict = {
            x : batch_x,
            y : batch_y
        }
        cost_, accuracy__, step_ = sess.run([cost, accuracy, global_step], feed_dict=feed_dict)
        validation_results.append((step_, cost_, accuracy__))
        
        # test performance
        batch_x = mnist.test.images
        batch_y = mnist.test.labels
        batch_x = batch_x[:, idx_permute]        
        batch_x = batch_x.reshape([-1, n_steps, input_dims])
        feed_dict = {
            x : batch_x,
            y : batch_y
        }
        cost_, accuracy_, step_ = sess.run([cost, accuracy, global_step], feed_dict=feed_dict)
        test_results.append((step_, cost_, accuracy_))        
        print ("========> Validation Accuarcy: " + "{:.6f}".format(accuracy__) \
        + ", Testing Accuarcy: " + "{:.6f}".format(accuracy_)) 
    step += 1

Iter 100, Minibatch Loss: 2.391761, Training Accuracy: 0.117188
Iter 200, Minibatch Loss: 2.184969, Training Accuracy: 0.195312
Iter 300, Minibatch Loss: 1.993967, Training Accuracy: 0.265625
Iter 400, Minibatch Loss: 2.125412, Training Accuracy: 0.171875
Iter 500, Minibatch Loss: 2.046483, Training Accuracy: 0.257812
Iter 600, Minibatch Loss: 1.843802, Training Accuracy: 0.335938
Iter 700, Minibatch Loss: 2.030382, Training Accuracy: 0.234375
Iter 800, Minibatch Loss: 1.801108, Training Accuracy: 0.398438
Iter 900, Minibatch Loss: 1.880221, Training Accuracy: 0.328125
Iter 1000, Minibatch Loss: 1.723603, Training Accuracy: 0.421875
Iter 1100, Minibatch Loss: 1.583453, Training Accuracy: 0.468750
Iter 1200, Minibatch Loss: 1.724710, Training Accuracy: 0.351562
Iter 1300, Minibatch Loss: 1.574113, Training Accuracy: 0.445312
Iter 1400, Minibatch Loss: 1.601763, Training Accuracy: 0.453125
Iter 1500, Minibatch Loss: 1.499128, Training Accuracy: 0.507812
Iter 1600, Minibatch Loss: 1.48830

Iter 12500, Minibatch Loss: 0.464938, Training Accuracy: 0.851562
Iter 12600, Minibatch Loss: 0.289597, Training Accuracy: 0.898438
Iter 12700, Minibatch Loss: 0.235080, Training Accuracy: 0.945312
Iter 12800, Minibatch Loss: 0.365599, Training Accuracy: 0.890625
Iter 12900, Minibatch Loss: 0.386882, Training Accuracy: 0.890625
Iter 13000, Minibatch Loss: 0.366081, Training Accuracy: 0.906250
Iter 13100, Minibatch Loss: 0.355104, Training Accuracy: 0.906250
Iter 13200, Minibatch Loss: 0.524092, Training Accuracy: 0.851562
Iter 13300, Minibatch Loss: 0.396775, Training Accuracy: 0.890625
Iter 13400, Minibatch Loss: 0.280340, Training Accuracy: 0.929688
Iter 13500, Minibatch Loss: 0.278650, Training Accuracy: 0.914062
Iter 13600, Minibatch Loss: 0.283853, Training Accuracy: 0.906250
Iter 13700, Minibatch Loss: 0.213493, Training Accuracy: 0.945312
Iter 13800, Minibatch Loss: 0.199230, Training Accuracy: 0.937500
Iter 13900, Minibatch Loss: 0.306694, Training Accuracy: 0.890625
Iter 14000

Iter 24800, Minibatch Loss: 0.209327, Training Accuracy: 0.929688
Iter 24900, Minibatch Loss: 0.175150, Training Accuracy: 0.937500
Iter 25000, Minibatch Loss: 0.184542, Training Accuracy: 0.953125
========> Validation Accuarcy: 0.923200, Testing Accuarcy: 0.922300
Iter 25100, Minibatch Loss: 0.105943, Training Accuracy: 0.968750
Iter 25200, Minibatch Loss: 0.102153, Training Accuracy: 0.968750
Iter 25300, Minibatch Loss: 0.201609, Training Accuracy: 0.945312
Iter 25400, Minibatch Loss: 0.231106, Training Accuracy: 0.953125
Iter 25500, Minibatch Loss: 0.150856, Training Accuracy: 0.953125
Iter 25600, Minibatch Loss: 0.203339, Training Accuracy: 0.937500
Iter 25700, Minibatch Loss: 0.162278, Training Accuracy: 0.960938
Iter 25800, Minibatch Loss: 0.117884, Training Accuracy: 0.960938
Iter 25900, Minibatch Loss: 0.166828, Training Accuracy: 0.960938
Iter 26000, Minibatch Loss: 0.220446, Training Accuracy: 0.929688
Iter 26100, Minibatch Loss: 0.286838, Training Accuracy: 0.914062
Iter 262